In [1]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [16]:
from os import listdir
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
import librosa
import soundfile

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8940430260402548310
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4559371323093208819
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5810552832
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5416370338252114173
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 1660599460387681269
physical_device_desc: "device: XLA_GPU device"
]


In [4]:
def sample_array(data, length):
    input_samples = list()
    for i in range(0,len(data), length):
        sample = data[i:i+length]
        if len(sample) != length:
            sample = np.pad(sample, (0, length - len(sample)), 'constant')
        input_samples.append(sample)
    return np.asarray(input_samples).reshape(len(input_samples), length, 1)

In [5]:
input_files = sorted(listdir('samples/combined-samples'))
output_files = sorted(listdir('samples/voice-samples'))
break_length = 200

In [6]:
input_samples = list()
for index, i in enumerate(input_files[0:100]):
    input_loaded = librosa.load("samples/combined-samples/{}".format(i))
    for i in range(0,len(input_loaded[0]), break_length):
        sample = input_loaded[0][i:i+break_length]
        if len(sample) != break_length:
            sample = np.pad(sample, (0, break_length - len(sample)), 'constant')
        input_samples.append(sample)

x_train = np.asarray(input_samples).reshape(len(input_samples), break_length, 1)
print(x_train.shape)

(48500, 200, 1)


In [7]:
output_samples = list()
for index, i in enumerate(output_files[0:100]):
    output_loaded = librosa.load("samples/voice-samples/{}".format(i))
    for i in range(0,len(output_loaded[0]), break_length):
        sample = output_loaded[0][i:i+break_length]
        if len(sample) != break_length:
            sample = np.pad(sample, (0, break_length - len(sample)), 'constant')
        output_samples.append(sample)
        
#output_samples = list()
#for i in range(0, len(output_loaded[0]), break_length):
#    sample = output_loaded[0][i:i+break_length]
#    if len(sample) != break_length:
#        sample = np.pad(sample, (0, break_length - len(sample)), 'constant')
#    input_samples.append(sample)
#    output_samples.append(sample)

y_train = np.asarray(output_samples)
print(y_train.shape)

(48500, 200)


In [8]:
print(x_train.shape)
print(y_train.shape)

(48500, 200, 1)
(48500, 200)


In [31]:
hidden_size = 200
model = Sequential()

model.add(LSTM(hidden_size, batch_input_shape=(500, break_length, 1), return_sequences=True, stateful=True))
model.add(LSTM(hidden_size, return_sequences=True, stateful=True))
model.add(LSTM(hidden_size, stateful=True))
model.add(Dense(200, activation='relu'))
model.compile(loss='binary_crossentropy', optimizer='adadelta')

In [33]:
model.fit(x_train, y_train, batch_size=500, epochs=1, shuffle=False)

Epoch 1/1
48500/48500 [==============================] - 96s 2ms/step - loss: -0.0041


In [19]:
test_file = 'samples/combined-samples/combined1024-2.wav'
test_loaded = librosa.load(test_file)
test_output = sample_array(test_loaded[0], 200)
print(test_output.shape)

(670, 200, 1)


In [20]:
output = model.predict(test_output)

In [21]:
soundfile.write('test_output.wav', output.flatten(), 44100)